In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('aita_clean.csv')

In [3]:
data.head()

,id,timestamp,title,body,edited,verdict,score,num_comments,is_asshole
0,1ytxov,1.393279e+09,[AITA] I wrote an explanation in TIL and came ...,[Here is the post in question](http://www.redd...,False,asshole,52,13.0,1
1,1yu29c,1.393281e+09,[AITA] Threw my parent's donuts away,"My parents are diabetic, morbidly obese, and a...",1393290576.0,asshole,140,27.0,1
2,1yu8hi,1.393285e+09,I told a goth girl she looked like a clown.,I was four.,False,not the asshole,74,15.0,0
3,1yuc78,1.393287e+09,[AItA]: Argument I had with another redditor i...,http://www.reddit.com/r/HIMYM/comments/1vvfkq/...,1393286962.0,everyone sucks,22,3.0,1
4,1yueqb,1.393288e+09,[AITA] I let my story get a little long and bo...,NaN,False,not the asshole,6,4.0,0


In [4]:
import nltk
from nltk.corpus import wordnet 

In [5]:
#nltk.download()

In [6]:
import pandas as pd

In [7]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

In [8]:
def cleaners(sentence):
    sentence = sentence.lower()
    #words = sentence.split()
    return sentence

In [9]:
from nltk.corpus import wordnet
from nltk import pos_tag

In [10]:
lemmatizer.lemmatize('corpora')

'corpus'

In [11]:
#data['lower_split'] = data['body'].astype(str).apply(lambda x : cleaners(x))

In [12]:
#text = word_tokenize('Rock Cutting Faster Going Corpora')
def postag_n_lemat(text):
    text = text.lower()
    posts = pos_tag(text.split())
    new_sent = []
    for i in range (0, len(posts)):
        #print(posts[i])        
        word = posts[i][0]
        tag = posts[i][1]
        if tag.startswith('R'):
            new_sent.append(lemmatizer.lemmatize(word,'r'))
        elif tag.startswith('N'):
            new_sent.append(lemmatizer.lemmatize(word,'n'))
        elif tag.startswith('V'):
            new_sent.append(lemmatizer.lemmatize(word,'v'))
        elif tag.startswith('J'):
            new_sent.append(lemmatizer.lemmatize(word, 'a'))
        else:
            new_sent.append(lemmatizer.lemmatize(word))
    joined = ' '.join(word for word in new_sent)
    return joined
    #nltk.pos_tag(text)
    

In [13]:
postag_n_lemat('Where are the rocks going in their paying and parking in the parked spot with better Corpora')

'where be the rock go in their pay and park in the parked spot with good corpus'

In [14]:
data['lemats'] = data['body'].astype(str).apply(lambda x : postag_n_lemat(x))

In [15]:
data.head()

,id,timestamp,title,body,edited,verdict,score,num_comments,is_asshole,lemats
0,1ytxov,1.393279e+09,[AITA] I wrote an explanation in TIL and came ...,[Here is the post in question](http://www.redd...,False,asshole,52,13.0,1,[here be the post in question](http://www.redd...
1,1yu29c,1.393281e+09,[AITA] Threw my parent's donuts away,"My parents are diabetic, morbidly obese, and a...",1393290576.0,asshole,140,27.0,1,"my parent be diabetic, morbidly obese, and add..."
2,1yu8hi,1.393285e+09,I told a goth girl she looked like a clown.,I was four.,False,not the asshole,74,15.0,0,i be four.
3,1yuc78,1.393287e+09,[AItA]: Argument I had with another redditor i...,http://www.reddit.com/r/HIMYM/comments/1vvfkq/...,1393286962.0,everyone sucks,22,3.0,1,http://www.reddit.com/r/himym/comments/1vvfkq/...
4,1yueqb,1.393288e+09,[AITA] I let my story get a little long and bo...,NaN,False,not the asshole,6,4.0,0,nan


In [16]:
import re

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction import text

In [19]:
my_stop_words = text.ENGLISH_STOP_WORDS.union(['way', 'didn','things', 'say', 'think', 'said', 've', 'want', 'know', 'feel', 'really', 'don', 'just', 'like'])

In [20]:
my_stop_words

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

In [22]:
tfidf_mystop = TfidfVectorizer(max_df = .7, min_df = 20, stop_words = my_stop_words)
dtm_mystop = tfidf_mystop.fit_transform(data['lemats'].values.astype('U'))
dtm_mystop

<97628x13972 sparse matrix of type '<class 'numpy.float64'>'
	with 7889824 stored elements in Compressed Sparse Row format>

In [23]:
nmf_model2 = NMF(n_components = 10)
nmf_model2.fit(dtm_mystop)
for index, topic in enumerate(nmf_model2.components_):
  print(f"The top 15 words for topic#{index}")
  print([tfidf_mystop.get_feature_names()[i] for i in topic.argsort()[-15:]])
  #print('\n')

The top 15 words for topic#0
['stop', 'look', 'x200b', 'girl', 'time', 'relationship', 'people', 'guy', 'start', 'ask', 'thing', 'make', 'try', 'talk', 'tell']


The top 15 words for topic#1
['old', 'aunt', 'grandma', 'live', 'house', 'year', 'father', 'tell', 'mother', 'family', 'parent', 'brother', 'sister', 'dad', 'mom']


The top 15 words for topic#2
['yard', 'care', 'house', 'vet', 'leash', 'neighbor', 'owner', 'cat', 'animal', 'bark', 'pet', 'walk', 'puppy', 'dogs', 'dog']


The top 15 words for topic#3
['afford', 'account', 'cost', 'use', 'gift', 'live', 'help', 'ask', 'save', 'job', 'month', 'rent', 'buy', 'money', 'pay']


The top 15 words for topic#4
['night', 'meet', 'tell', 'girl', 'birthday', 'plan', 'close', 'boyfriend', 'invite', 'friends', 'best', 'hang', 'party', 'group', 'friend']


The top 15 words for topic#5
['christmas', 'parent', 'father', 'old', 'wedding', 'baby', 'year', 'mother', 'child', 'son', 'daughter', 'husband', 'kid', 'family', 'wife']


The top 15 word

In [24]:
topic_results2 = nmf_model2.transform(dtm_mystop)
data['MystopTopic_10'] = topic_results2.argmax(axis = 1)
data['MystopTopic_10'].value_counts()

0    19361
5    14638
4    10861
8    10210
6     9507
1     9111
3     8447
9     7235
7     5132
2     3126
Name: MystopTopic_10, dtype: int64

In [26]:
nmf_model3 = NMF(n_components = 100)
nmf_model3.fit(dtm_mystop)
for index, topic in enumerate(nmf_model3.components_):
  print(f"The top 10 words for topic#{index}")
  print([tfidf_mystop.get_feature_names()[i] for i in topic.argsort()[-10:]])
  #print('\n')

The top 10 words for topic#0
['read', 'texted', 'email', 'block', 'contact', 'reply', 'respond', 'text', 'send', 'message']
The top 10 words for topic#1
['care', 'mad', 'aunt', 'live', 'yell', 'divorce', 'step', 'stepdad', 'grandma', 'mom']
The top 10 words for topic#2
['neighbor', 'vet', 'leash', 'pet', 'animal', 'owner', 'bark', 'puppy', 'dogs', 'dog']
The top 10 words for topic#3
['months', 'sign', 'couple', 'deposit', 'landlord', 'place', 'ago', 'lease', 'month', 'rent']
The top 10 words for topic#4
['insurance', 'debt', 'payment', 'loan', '50', 'half', 'split', 'offer', 'cost', 'pay']
The top 10 words for topic#5
['funeral', 'vacation', 'extended', 'sibling', 'holiday', 'visit', 'close', 'thanksgiving', 'member', 'family']
The top 10 words for topic#6
['dishes', 'laundry', 'cleaning', 'sink', 'mess', 'bathroom', 'kitchen', 'wash', 'dish', 'clean']
The top 10 words for topic#7
['new', 'cars', 'road', 'vehicle', 'driver', 'damage', 'gas', 'accident', 'insurance', 'car']
The top 10 w

['wine', 'water', 'bottle', 'beer', 'night', 'alcohol', 'bar', 'drinking', 'drunk', 'drink']
The top 10 words for topic#69
['garage', 'lot', 'block', 'neighbor', 'driveway', 'space', 'street', 'parking', 'spot', 'park']
The top 10 words for topic#70
['outside', 'close', 'loud', 'hear', 'bathroom', 'knock', 'neighbor', 'lock', 'open', 'door']
The top 10 words for topic#71
['long', 'black', 'color', 'shower', 'look', 'haircut', 'shave', 'dye', 'cut', 'hair']
The top 10 words for topic#72
['lease', 'neighbor', 'complex', 'living', 'bedroom', 'building', 'city', 'place', 'live', 'apartment']
The top 10 words for topic#73
['word', 'password', 'change', 'toilet', 'computer', 'card', 'laptop', 'bathroom', 'account', 'use']
The top 10 words for topic#74
['festival', 'book', 'movie', 'plane', 'sell', 'event', 'band', 'tickets', 'concert', 'ticket']
The top 10 words for topic#75
['dates', 'week', 'months', 'woman', 'tinder', 'month', 'dating', 'start', 'meet', 'date']
The top 10 words for topic#

In [28]:
topic_results3 = nmf_model3.transform(dtm_mystop)
data['MystopTopic_100'] = topic_results3.argmax(axis = 1)
#data['MystopTopic_100'].value_counts()

In [30]:
nmf_model7 = NMF(n_components = 7)
nmf_model7.fit(dtm_mystop)
for index, topic in enumerate(nmf_model7.components_):
  print(f"The top 7 words for topic#{index}")
  print([tfidf_mystop.get_feature_names()[i] for i in topic.argsort()[-7:]])
  #print('\n')
topic_results7 = nmf_model7.transform(dtm_mystop)
data['MystopTopic_7'] = topic_results7.argmax(axis = 1)

The top 7 words for topic#0
['car', 'ask', 'tell', 'job', 'time', 'day', 'work']
The top 7 words for topic#1
['family', 'parent', 'tell', 'brother', 'sister', 'dad', 'mom']
The top 7 words for topic#2
['bark', 'animal', 'pet', 'walk', 'puppy', 'dogs', 'dog']
The top 7 words for topic#3
['month', 'job', 'car', 'rent', 'buy', 'money', 'pay']
The top 7 words for topic#4
['date', 'guy', 'group', 'girl', 'talk', 'tell', 'friend']
The top 7 words for topic#5
['year', 'son', 'daughter', 'husband', 'kid', 'wife', 'family']
The top 7 words for topic#6
['night', 'bed', 'cat', 'house', 'roommate', 'sleep', 'room']


In [31]:
nmf_model250 = NMF(n_components = 250)
nmf_model250.fit(dtm_mystop)
for index, topic in enumerate(nmf_model250.components_):
  print(f"The top 7 words for topic#{index}")
  print([tfidf_mystop.get_feature_names()[i] for i in topic.argsort()[-7:]])
  #print('\n')
topic_results250 = nmf_model250.transform(dtm_mystop)
data['MystopTopic_250'] = topic_results250.argmax(axis = 1)

The top 7 words for topic#0
['honestly', 'basically', 'kinda', 'probably', 'kind', 'sure', 'pretty']
The top 7 words for topic#1
['hate', 'fight', 'anyways', 'divorce', 'yell', 'stepdad', 'mom']
The top 7 words for topic#2
['pet', 'animal', 'owner', 'bark', 'puppy', 'dogs', 'dog']
The top 7 words for topic#3
['debt', 'owe', 'insurance', 'payment', 'loan', 'cost', 'pay']
The top 7 words for topic#4
['agree', 'cover', 'fair', 'cost', '50', 'split', 'half']
The top 7 words for topic#5
['happy', 'kids', 'years', 'married', 'marriage', 'divorce', 'marry']
The top 7 words for topic#6
['flush', 'floor', 'pee', 'stall', 'paper', 'toilet', 'bathroom']
The top 7 words for topic#7
['vehicle', 'cars', 'damage', 'gas', 'accident', 'insurance', 'car']
The top 7 words for topic#8
['schedule', 'able', 'busy', 'worker', 'working', 'hard', 'work']
The top 7 words for topic#9
['pan', 'meat', 'chicken', 'kitchen', 'cooking', 'meal', 'cook']
The top 7 words for topic#10
['refuse', 'bil', 'nephew', 'si', 't

['delivery', 'waiter', 'restaurant', 'waitress', 'server', 'service', 'tip']
The top 7 words for topic#88
['emotionally', 'great', 'toxic', 'open', 'abusive', 'distance', 'relationship']
The top 7 words for topic#89
['set', 'prom', 'months', 'dates', 'tinder', 'dating', 'date']
The top 7 words for topic#90
['saw', 'gathering', 'grow', 'drama', 'relative', 'cousins', 'cousin']
The top 7 words for topic#91
['weeks', 'drug', 'hasn', 'months', 'haven', 'years', 'past']
The top 7 words for topic#92
['kick', 'fear', 'pass', 'nana', 'grandad', 'funeral', 'nan']
The top 7 words for topic#93
['completely', 'tie', 'block', 'number', 'reach', 'cut', 'contact']
The top 7 words for topic#94
['aita', 'whilst', 'holiday', 'discuss', 'quite', 'partners', 'partner']
The top 7 words for topic#95
['friday', 'twice', 'schedule', 'notice', 'weeks', 'ago', 'week']
The top 7 words for topic#96
['maybe', 'title', 'wonder', 'recently', 'reddit', 'situation', 'asshole']
The top 7 words for topic#97
['cool', 'ou

['aren', 'closer', 'consider', 'grow', 'friendship', 'super', 'close']
The top 7 words for topic#173
['entire', 'drug', 'hard', 'real', 'grow', 'happy', 'life']
The top 7 words for topic#174
['tables', 'waitress', 'bar', 'lunch', 'server', 'restaurant', 'table']
The top 7 words for topic#175
['diet', 'hungry', 'eating', 'pizza', 'meal', 'lunch', 'eat']
The top 7 words for topic#176
['wait', 'minutes', '15', '20', '30', '10', 'minute']
The top 7 words for topic#177
['semester', 'uni', 'final', 'university', 'student', 'exam', 'study']
The top 7 words for topic#178
['fiancés', 'propose', 'engagement', 'engaged', 'future', 'engage', 'fiancé']
The top 7 words for topic#179
['towel', 'morning', 'pee', 'wash', 'showers', 'bridal', 'shower']
The top 7 words for topic#180
['check', 'essay', 'paper', 'note', 'read', 'letter', 'write']
The top 7 words for topic#181
['quit', 'meeting', 'coworkers', 'worker', 'employee', 'boss', 'bos']
The top 7 words for topic#182
['situation', 'summer', 'happen'

In [32]:
aita_labels = data[['id','MystopTopic_10','MystopTopic_100','MystopTopic_250']]

In [33]:
aita_labels.head()

,id,MystopTopic_10,MystopTopic_100,MystopTopic_250
0,1ytxov,0,76,0
1,1yu29c,9,91,14
2,1yu8hi,0,0,0
3,1yuc78,0,76,222
4,1yueqb,1,90,92


In [34]:
aita_labels.to_csv('aita_labels10100250.csv', index = False)